# Pruebas para verificar el cumplimiento del objetivo específico B

## Configuración Inicial y Autenticación

In [1]:
import os
import requests
import threading
import pandas as pd
import random
from dotenv import load_dotenv

In [2]:
# function to get the access token
def get_access_token(client_id, client_secret):
    token_url = "https://accounts.spotify.com/api/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    try:
        response = requests.post(token_url, headers=headers, data=data)
        response.raise_for_status()

        print('✅ Token loaded.')

        return response.json()["access_token"]
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener el token de acceso: {e}")
        return None

In [3]:
# configuration
load_dotenv()

CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')

# get the access token
ACCESS_TOKEN = get_access_token(CLIENT_ID, CLIENT_SECRET)
if not ACCESS_TOKEN:
    print("No se pudo obtener el token de acceso. Las pruebas no pueden continuar.")
    exit()

HEADERS = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "Content-Type": "application/json"
}

ENDPOINTS_TO_TEST = [
    # search
    "https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album",
    # get album
    "https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy",
    # get artist
    "https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg",
    # get playlist
    "https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n",
    # get track
    "https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl",
    # TODO Puedes añadir más endpoints aquí
]

# variables to save the results
results_lock = threading.Lock()
concurrent_test_results = []
concurrent_test_individual_results = {
    'user_ids': [],
    'endpoints': [],
    'status_codes': [],
    'latencies': [],
    'times': [] # Para registrar el tiempo en que se realizó la petición
}

✅ Token loaded.


## Funcion de concurrencia para realizar las pruebas

In [4]:
import time


def user_simulation_task(user_id, endpoint, headers, num_requests_per_user):
    """
    Simula el comportamiento de un único usuario haciendo peticiones a un endpoint.
    """

    user_results = {
        'user_id': user_id,
        'endpoint': endpoint,
        'total_requests': 0,
        'successful_requests': 0,
        'error_4xx_requests': 0,
        'error_5xx_requests': 0,
        'total_response_time': 0,
        'latencies': [], # Para registrar tiempos de respuesta individuales
        'errors_occurred': [], # Para registrar códigos de error
        'test_duration': 0
    }

    user_ids = []
    endpoints = []
    status_codes = []
    latencies = []
    times = []

    # test info
    test_start_time = time.time()
    print(f"✴️ Usuario {user_id} - Starting requests to {endpoint}. - Test started at {test_start_time}")

    # do requests for the given amount
    for i in range(num_requests_per_user):

        start_time = time.time()

        try:
            # request
            response = requests.get(endpoint, headers=headers)

            # ☑️ save the data to study

            #   save the whole test results
            duration = time.time() - start_time
            user_results['total_requests'] += 1
            user_results['total_response_time'] += duration
            user_results['latencies'].append(duration)
            status_code = response.status_code

            #   save the individual test results
            user_ids.append(user_id)
            endpoints.append(endpoint)
            status_codes.append(status_code)
            latencies.append(duration)
            times.append(time.time()) # save the time in floating point

            if 200 <= status_code < 300: # if request successful
                user_results['successful_requests'] += 1

            elif 400 <= status_code < 500: # if request of kind 4xx
                user_results['error_4xx_requests'] += 1
                user_results['errors_occurred'].append(status_code)

                if status_code == 429: # if request of kind 429 (to many requests)
                    print(f"Usuario {user_id} - Alerta: Recibido 429 (Too Many Requests).")

                    #retry_after = response.headers.get('Retry-After')
                    # if retry_after:
                    #     wait_time = int(retry_after) + 1
                    #     print(f"Usuario {user_id} - Esperando {wait_time}s antes de reintentar.")
                    #     time.sleep(wait_time) # Esperar antes de la próxima petición

            elif 500 <= status_code < 600:
                user_results['error_5xx_requests'] += 1
                user_results['errors_occurred'].append(status_code)
                print(f"Usuario {user_id} - Error 5xx ({status_code}) en {endpoint}.")

        except requests.exceptions.RequestException as e:

            user_results['total_requests'] += 1 # Contar también las peticiones con excepción
            user_results['errors_occurred'].append(str(e)) # Registrar la excepción
            print(f"Usuario {user_id} - Excepción de red para {endpoint}: {e}")

    test_finish_time = time.time()
    test_duration = test_finish_time - test_start_time

    # save the test duration
    user_results['test_duration'] = test_duration

    print(f"✅ User {user_id} - Finished requests to {endpoint}. - Test finished at {test_finish_time}. Duration: {test_duration:.2f}s")

    # add the results to the global results
    with results_lock:
        # save the whole test results
        concurrent_test_results.append(user_results)
        # save the individual test results
        concurrent_test_individual_results['user_ids'].extend(user_ids)
        concurrent_test_individual_results['endpoints'].extend(endpoints)
        concurrent_test_individual_results['status_codes'].extend(status_codes)
        concurrent_test_individual_results['latencies'].extend(latencies)
        concurrent_test_individual_results['times'].extend(times)


## Orquestacion de usuarios concurrentes

Simulamos multiples usuarios de forma concurrente para probar los endpoints de la API de Spotify.

In [5]:
def simulate_concurrent_users(num_users, num_requests_per_user, endpoints_to_test):
    """
    Orquesta la simulación de múltiples usuarios concurrentes.

    Args:
        num_users (int): El número de usuarios a simular.
        num_requests_per_user (int): El número de peticiones que cada usuario hará.
        endpoints_to_test (list): Lista de endpoints a los que los usuarios harán peticiones.
                                  Los usuarios se distribuirán entre estos endpoints.
    """

    print(f"\n--- Simulation started with {num_users} concurrent users ---")
    print(f"Each user will do {num_requests_per_user} requests.\n")

    threads = []

    # create a thread for each user
    endpoint_index = 0

    for i in range(num_users):
        # select the endpoint for the user
        selected_endpoint = endpoints_to_test[endpoint_index % len(endpoints_to_test)]

        # create the thread
        thread = threading.Thread(
            target=user_simulation_task,
            args=(i + 1, selected_endpoint, HEADERS, num_requests_per_user)
        )
        threads.append(thread)
        thread.start() # Inicia el hilo
        endpoint_index += 1

        # wait i little for the next thread
        time.sleep(random.uniform(0.01, 0.1))

    # wait to each thread to finish
    for thread in threads:
        thread.join()

    print("\n--- Simulation finished ---")

# Objetivo B

Hacemos las pruebas base para cumplir con el objetivo A. Vamos a probar todos los endpoints que declaramos que son los servicios a estudiar, y vamos a tomar todas las mediciones para luego poder responder a las metricas establecidas.

In [6]:
simulate_concurrent_users(
    num_users=30,
    num_requests_per_user=30,
    endpoints_to_test=ENDPOINTS_TO_TEST
)


--- Simulation started with 30 concurrent users ---
Each user will do 30 requests.

✴️ Usuario 1 - Starting requests to https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album. - Test started at 1747954400.5652652
✴️ Usuario 2 - Starting requests to https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy. - Test started at 1747954400.6237686
✴️ Usuario 3 - Starting requests to https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg. - Test started at 1747954400.7012658
✴️ Usuario 4 - Starting requests to https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n. - Test started at 1747954400.7408156
✴️ Usuario 5 - Starting requests to https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl. - Test started at 1747954400.8166826
✴️ Usuario 6 - Starting requests to https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album. - Test started at 1747954400.8623116
✴️ Usuario 7 - Starting requests to

## Procesa los resultados

In [7]:
def process_concurrent_results():
    """
    Consolida y analiza los resultados de la simulación de usuarios concurrentes.
    """
    if not concurrent_test_results:
        print("No hay resultados para procesar de la simulación concurrente.")
        return

    df_concurrent = pd.DataFrame(concurrent_test_results)
    df_concurrent_2 = pd.DataFrame(concurrent_test_individual_results)

    # global calculates
    global_total_requests = df_concurrent['total_requests'].sum()
    global_successful_requests = df_concurrent['successful_requests'].sum()
    global_error_4xx_requests = df_concurrent['error_4xx_requests'].sum()
    global_error_5xx_requests = df_concurrent['error_5xx_requests'].sum()
    global_success_rate = (global_successful_requests / global_total_requests) * 100 if global_total_requests > 0 else 0
    global_error_4xx_rate = (global_error_4xx_requests / global_total_requests) * 100 if global_total_requests > 0 else 0
    global_error_5xx_rate = (global_error_5xx_requests / global_total_requests) * 100 if global_total_requests > 0 else 0

    # global results
    print("\n--- Resumen General de la Simulación Concurrente ---")
    print(f"Total de usuarios simulados: {len(df_concurrent)}")
    print(f"Peticiones totales realizadas: {df_concurrent['total_requests'].sum()}")
    print(f"Peticiones exitosas totales: {df_concurrent['successful_requests'].sum()}")
    print(f"Errores 4xx totales: {df_concurrent['error_4xx_requests'].sum()}")
    print(f"Errores 5xx totales: {df_concurrent['error_5xx_requests'].sum()}")
    print(f"Tasa de éxito global: {global_success_rate:.2f}%")
    print(f"Tasa de error 4xx global: {global_error_4xx_rate:.2f}%")
    print(f"Tasa de error 5xx global: {global_error_5xx_rate:.2f}%")

    # analysis by endpoint
    agg_by_endpoint = df_concurrent.groupby('endpoint').agg(
        total_requests=('total_requests', 'sum'),
        successful_requests=('successful_requests', 'sum'),
        error_4xx_requests=('error_4xx_requests', 'sum'),
        error_5xx_requests=('error_5xx_requests', 'sum'),
        avg_latency=('latencies', lambda x: pd.Series([item for sublist in x for item in sublist]).mean()),
        num_users=('user_id', 'nunique')
    ).reset_index()

    agg_by_endpoint['success_rate'] = (agg_by_endpoint['successful_requests'] / agg_by_endpoint['total_requests']) * 100
    agg_by_endpoint['error_4xx_rate'] = (agg_by_endpoint['error_4xx_requests'] / agg_by_endpoint['total_requests']) * 100
    agg_by_endpoint['error_5xx_rate'] = (agg_by_endpoint['error_5xx_requests'] / agg_by_endpoint['total_requests']) * 100

    return df_concurrent, df_concurrent_2, agg_by_endpoint

In [8]:
# process the results
df_global_results, df_individual_results, df_by_endpoint = process_concurrent_results()


--- Resumen General de la Simulación Concurrente ---
Total de usuarios simulados: 30
Peticiones totales realizadas: 900
Peticiones exitosas totales: 101
Errores 4xx totales: 799
Errores 5xx totales: 0
Tasa de éxito global: 11.22%
Tasa de error 4xx global: 88.78%
Tasa de error 5xx global: 0.00%


In [9]:
print('\nGlobal results\n')
df_global_results


Global results



,user_id,endpoint,total_requests,successful_requests,error_4xx_requests,error_5xx_requests,total_response_time,latencies,errors_occurred,test_duration
0,3,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,8,22,0,4.664892,"[0.17102408409118652, 0.1700127124786377, 0.17...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",4.666938
1,8,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,7,23,0,4.614571,"[0.16965413093566895, 0.17777156829833984, 0.1...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",4.615580
2,13,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,5,25,0,4.613210,"[0.19733262062072754, 0.1843714714050293, 0.20...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",4.614215
3,12,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,30,5,25,0,4.723556,"[0.21969032287597656, 0.2014477252960205, 0.19...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",4.724555
4,4,https://api.spotify.com/v1/playlists/3cEYpjA9o...,30,6,24,0,5.137799,"[0.2326974868774414, 0.1989271640777588, 0.214...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",5.139807
5,11,https://api.spotify.com/v1/search?q=remaster%2...,30,4,26,0,5.032715,"[0.3283545970916748, 0.2941768169403076, 0.299...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",5.033719
6,25,https://api.spotify.com/v1/tracks/11dFghVXANMl...,30,1,29,0,4.444602,"[0.17282581329345703, 0.14597535133361816, 0.1...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",4.447129
7,19,https://api.spotify.com/v1/playlists/3cEYpjA9o...,30,3,27,0,4.960088,"[0.28289175033569336, 0.21810340881347656, 0.2...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",4.961844
8,6,https://api.spotify.com/v1/search?q=remaster%2...,30,4,26,0,5.635877,"[0.30411601066589355, 0.30491137504577637, 0.2...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",5.639389
9,28,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,0,30,0,4.373225,"[0.14745736122131348, 0.15002989768981934, 0.1...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",4.374229


In [10]:
print('\nIndividual results\n')
df_individual_results


Individual results



,user_ids,endpoints,status_codes,latencies,times
0,3,https://api.spotify.com/v1/artists/0TnOYISbd1X...,200,0.171024,1.747954e+09
1,3,https://api.spotify.com/v1/artists/0TnOYISbd1X...,200,0.170013,1.747954e+09
2,3,https://api.spotify.com/v1/artists/0TnOYISbd1X...,200,0.171032,1.747954e+09
3,3,https://api.spotify.com/v1/artists/0TnOYISbd1X...,200,0.190780,1.747954e+09
4,3,https://api.spotify.com/v1/artists/0TnOYISbd1X...,200,0.175090,1.747954e+09
...,...,...,...,...,...
895,27,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,429,0.145296,1.747954e+09
896,27,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,429,0.146013,1.747954e+09
897,27,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,429,0.453149,1.747954e+09
898,27,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,429,0.148253,1.747954e+09


In [11]:
print('\nResults by endpoint\n')
df_by_endpoint


Results by endpoint



,endpoint,total_requests,successful_requests,error_4xx_requests,error_5xx_requests,avg_latency,num_users,success_rate,error_4xx_rate,error_5xx_rate
0,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,180,20,160,0,0.189261,6,11.111111,88.888889,0.0
1,https://api.spotify.com/v1/artists/0TnOYISbd1X...,180,26,154,0,0.166371,6,14.444444,85.555556,0.0
2,https://api.spotify.com/v1/playlists/3cEYpjA9o...,180,21,159,0,0.177430,6,11.666667,88.333333,0.0
3,https://api.spotify.com/v1/search?q=remaster%2...,180,14,166,0,0.182653,6,7.777778,92.222222,0.0
4,https://api.spotify.com/v1/tracks/11dFghVXANMl...,180,20,160,0,0.185260,6,11.111111,88.888889,0.0


## Guardamos los resultados base

In [12]:
df_global_results.to_csv('./files/obj-B/global_results.csv', index=False)
df_individual_results.to_csv('./files/obj-B/individual_results.csv', index=False)
df_by_endpoint.to_csv('./files/obj-B/endpoints_results.csv', index=False)